In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import re
import requests
import pandas as pd


In [19]:
# Scrapping the data from IMDB website
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get("https://www.imdb.com/chart/top/?ref_=nv_mv_250")

wait = WebDriverWait(driver, 10)

try:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    movies = soup.find_all('li', class_ = 'ipc-metadata-list-summary-item sc-4929eaf6-0 DLYcv cli-parent')
    print(len(movies))
finally:
    driver.quit()

250


In [21]:
#function to get title, href and movieid
def get_id_href(soup):
    movie=soup.find('a', attrs={'class':'ipc-title-link-wrapper'})
    title=''
    href=''
    movieid=''
    title=str(movie.text[3:].strip())
    href=movie['href']
    movieid = href.split('/')[2]
    return movieid,href,str(title)

In [23]:
#function to get year, duration and censorship
def get_year_time(soup):
    ytr = soup.find_all('span', attrs={'class':'sc-5bc66c50-6 OOdsw cli-title-metadata-item'})
    print('in function ytr',ytr)
    x = []
    for i in ytr:
        x.append(i.text)
    print('x:',x)
    
    try: 
        year = int(x[0])
    except:
        year = ''
    try:
        match = re.match(r'(?:(\d+)h)?\s*(?:(\d+)m)?', x[1])
        # Extract hours and minutes, defaulting to 0 if not found
        hours = int(match.group(1)) if match.group(1) else 0
        minutes = int(match.group(2)) if match.group(2) else 0
        # Convert the total time to minutes
        total_minutes = hours * 60 + minutes
        duration = total_minutes
    except:
        duration = ''
    try: 
        censorship = str(x[2])
    except:
        censorship = ''
    return year, duration, censorship

In [25]:
def get_ratings(soup):
    imdb_rating = float(soup.find('span', attrs={'class':'ipc-rating-star--rating'}).text)
    rating_count = soup.find('span', attrs={'class':'ipc-rating-star--voteCount'}).text.strip()
    match = re.match(r'\(?([\d\.]+)([MK]?)\)?', rating_count)
    number = float(match.group(1))  
    suffix = match.group(2)  
    if suffix == 'M':
        number = number  
    elif suffix == 'K':
        number = round(number / 1000,2)  
    else:
        number =  round(number / 1_000_000,2)  
    rating_count = float(number)
    return imdb_rating, rating_count

In [27]:
#function to get cast, director, tags
def get_movie_det(href, movie_data):
    url = "https://www.imdb.com"+href
    print(url)
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        print("Successfully fetched the webpage!")
        s = BeautifulSoup(response.content, "html.parser")
        time.sleep(5)
        try:
            tags = s.find_all('span',class_ = 'ipc-chip__text')
            movie_tags = [tag.text for tag in tags]
        except:
            movie_tags = []
        movie_data.get("Tags").append(movie_tags)

        try:
            summary = s.find('span', attrs={'data-testid':'plot-xl'}).text
        except:
            summary = ''
        movie_data.get("Summary").append(summary)
        
        try:
            crew = s.find('ul', class_ = 'ipc-metadata-list ipc-metadata-list--dividers-all title-pc-list ipc-metadata-list--baseAlt')
            crew_list = s.find_all('a', class_ ='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
        except:
            cre_list = []
            
        if len(crew_list)!=0:
            actors=[]
            directors=[]
            for i in crew_list:
                if 'dr' in i['href']:
                    # print(i.text)
                    directors.append(i.text)
                elif 'ov_st' in i['href']:
                    # print(i.text)
                    actors.append(i.text)
        movie_data.get('Director').append(directors)
        movie_data.get('Actors').append(actors)
                
    else:
        print(f"Failed to retrieve webpage. Status code: {response.status_code}")

In [29]:
movie_data = {'Movie_id':[],'Title':[],'Link':[],'Year':[],'Runtime':[],'Censorship':[],'Ratings':[],'Rating_count':[],'Tags':[],'Summary':[],'Director':[],'Actors':[]}
for i in range(len(movies)):
    id_href=get_id_href(movies[i])
    ytr=get_year_time(movies[i])
    rt = get_ratings(movies[i])
    movie_data.get('Movie_id').append(id_href[0])
    movie_data.get('Link').append(id_href[1])
    movie_data.get('Title').append(id_href[2])
    movie_data.get('Year').append(ytr[0])
    movie_data.get('Runtime').append(ytr[1])
    movie_data.get('Censorship').append(ytr[2])
    movie_data.get('Ratings').append(rt[0])
    movie_data.get('Rating_count').append(rt[1])
    get_movie_det(id_href[1],movie_data)
    
    
    


in function ytr [<span class="sc-5bc66c50-6 OOdsw cli-title-metadata-item">1994</span>, <span class="sc-5bc66c50-6 OOdsw cli-title-metadata-item">2h 22m</span>, <span class="sc-5bc66c50-6 OOdsw cli-title-metadata-item">R</span>]
x: ['1994', '2h 22m', 'R']
https://www.imdb.com/title/tt0111161/?ref_=chttp_t_1
Successfully fetched the webpage!
in function ytr [<span class="sc-5bc66c50-6 OOdsw cli-title-metadata-item">1972</span>, <span class="sc-5bc66c50-6 OOdsw cli-title-metadata-item">2h 55m</span>, <span class="sc-5bc66c50-6 OOdsw cli-title-metadata-item">R</span>]
x: ['1972', '2h 55m', 'R']
https://www.imdb.com/title/tt0068646/?ref_=chttp_t_2
Successfully fetched the webpage!
in function ytr [<span class="sc-5bc66c50-6 OOdsw cli-title-metadata-item">2008</span>, <span class="sc-5bc66c50-6 OOdsw cli-title-metadata-item">2h 32m</span>, <span class="sc-5bc66c50-6 OOdsw cli-title-metadata-item">PG-13</span>]
x: ['2008', '2h 32m', 'PG-13']
https://www.imdb.com/title/tt0468569/?ref_=chttp_t

In [31]:
print ("movie_id:", len(movie_data['Movie_id']))
print ("Title", len(movie_data['Title']))
print ("LINK", len(movie_data['Link']))
print ("Year", len(movie_data['Year']))
print ("Runtime", len(movie_data['Runtime']))
print ("Censorship", len(movie_data['Censorship']))
print ("Ratings", len(movie_data['Ratings']))
print ("Rating_count", len(movie_data['Rating_count']))



movie_id: 250
Title 250
LINK 250
Year 250
Runtime 250
Censorship 250
Ratings 250
Rating_count 250


In [33]:
import pandas as pd

movie_df=pd.DataFrame(movie_data)
movie_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Movie_id      250 non-null    object 
 1   Title         250 non-null    object 
 2   Link          250 non-null    object 
 3   Year          250 non-null    int64  
 4   Runtime       250 non-null    int64  
 5   Censorship    250 non-null    object 
 6   Ratings       250 non-null    float64
 7   Rating_count  250 non-null    float64
 8   Tags          250 non-null    object 
 9   Summary       250 non-null    object 
 10  Director      250 non-null    object 
 11  Actors        250 non-null    object 
dtypes: float64(2), int64(2), object(8)
memory usage: 23.6+ KB


In [35]:
movie_df.head(10)

,Movie_id,Title,Link,Year,Runtime,Censorship,Ratings,Rating_count,Tags,Summary,Director,Actors
0,tt0111161,The Shawshank Redemption,/title/tt0111161/?ref_=chttp_t_1,1994,142,R,9.3,3.00,"[Epic, Prison Drama, Drama, Back to top]",A banker convicted of uxoricide forms a friend...,"[Frank Darabont, Frank Darabont, Frank Darabont]","[Tim Robbins, Morgan Freeman, Bob Gunton, Tim ..."
1,tt0068646,The Godfather,/title/tt0068646/?ref_=chttp_t_2,1972,175,R,9.2,2.10,"[Epic, Gangster, Tragedy, Crime, Drama, Back t...",The aging patriarch of an organized crime dyna...,"[Francis Ford Coppola, Francis Ford Coppola, F...","[Marlon Brando, Al Pacino, James Caan, Marlon ..."
2,tt0468569,The Dark Knight,/title/tt0468569/?ref_=chttp_t_3,2008,152,PG-13,9.0,2.90,"[Action Epic, Epic, Superhero, Action, Crime, ...",When a menace known as the Joker wreaks havoc ...,"[Christopher Nolan, Christopher Nolan, Christo...","[Christian Bale, Heath Ledger, Aaron Eckhart, ..."
3,tt0071562,The Godfather Part II,/title/tt0071562/?ref_=chttp_t_4,1974,202,R,9.0,1.40,"[Epic, Gangster, Tragedy, Crime, Drama, Back t...",The early life and career of Vito Corleone in ...,"[Francis Ford Coppola, Francis Ford Coppola, F...","[Al Pacino, Robert De Niro, Robert Duvall, Al ..."
4,tt0050083,12 Angry Men,/title/tt0050083/?ref_=chttp_t_5,1957,96,Approved,9.0,0.89,"[Legal Drama, Psychological Drama, Crime, Dram...",The jury in a New York City murder trial is fr...,"[Sidney Lumet, Sidney Lumet, Sidney Lumet]","[Henry Fonda, Lee J. Cobb, Martin Balsam, Henr..."
5,tt0167260,The Lord of the Rings: The Return of the King,/title/tt0167260/?ref_=chttp_t_6,2003,201,PG-13,9.0,2.00,"[Adventure Epic, Epic, Fantasy Epic, Mountain ...",Gandalf and Aragorn lead the World of Men agai...,"[Peter Jackson, Peter Jackson, Peter Jackson]","[Elijah Wood, Viggo Mortensen, Ian McKellen, E..."
6,tt0108052,Schindler's List,/title/tt0108052/?ref_=chttp_t_7,1993,195,R,9.0,1.50,"[Docudrama, Epic, Historical Epic, Period Dram...","In German-occupied Poland during World War II,...","[Steven Spielberg, Steven Spielberg, Steven Sp...","[Liam Neeson, Ralph Fiennes, Ben Kingsley, Lia..."
7,tt0110912,Pulp Fiction,/title/tt0110912/?ref_=chttp_t_8,1994,154,R,8.9,2.30,"[Dark Comedy, Drug Crime, Gangster, Crime, Dra...","The lives of two mob hitmen, a boxer, a gangst...","[Quentin Tarantino, Quentin Tarantino, Quentin...","[John Travolta, Uma Thurman, Samuel L. Jackson..."
8,tt0120737,The Lord of the Rings: The Fellowship of the Ring,/title/tt0120737/?ref_=chttp_t_9,2001,178,PG-13,8.9,2.10,"[Adventure Epic, Epic, Fantasy Epic, Quest, Sw...",A meek Hobbit from the Shire and eight compani...,"[Peter Jackson, Peter Jackson, Peter Jackson]","[Elijah Wood, Ian McKellen, Orlando Bloom, Eli..."
9,tt0060196,"The Good, the Bad and the Ugly",/title/tt0060196/?ref_=chttp_t_10,1966,178,R,8.8,0.83,"[Desert Adventure, Quest, Spaghetti Western, W...",A bounty hunting scam joins two men in an unea...,"[Sergio Leone, Sergio Leone, Sergio Leone]","[Clint Eastwood, Eli Wallach, Lee Van Cleef, C..."


In [37]:
movie_df.to_excel('movies_metadata.xlsx', index = False)

In [73]:
#Scraping movie reviews 
import time

reviews_data1 = {'Movie_id':[],'review':[]}
for link in movie_data['Link']:
    movie_id=link.split('/')[2]
    url = "https://www.imdb.com"+link[:link.find('?')]+'reviews/?ref_=tt_urv_sm'  # Replace with the target URL
    print(url)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    
    
    # Check if the request was successful
    if response.status_code == 200:
        print("Successfully fetched the webpage!")
        s = BeautifulSoup(response.content, "html.parser")
        time.sleep(5) #change time if necessary
        reviews = s.find_all('div', class_='ipc-html-content-inner-div')
    
        for review in reviews:
            reviews_data1.get('Movie_id').append(movie_id)
            x=review.text
            reviews_data1.get('review').append(x[:x.find('\n',1)].lstrip('\n'))
    else:
        print(f"Failed to retrieve webpage. Status code: {response.status_code}")
    

https://www.imdb.com/title/tt0111161/reviews/?ref_=tt_urv_sm
Successfully fetched the webpage!
https://www.imdb.com/title/tt0068646/reviews/?ref_=tt_urv_sm
Successfully fetched the webpage!
https://www.imdb.com/title/tt0468569/reviews/?ref_=tt_urv_sm
Successfully fetched the webpage!
https://www.imdb.com/title/tt0071562/reviews/?ref_=tt_urv_sm
Successfully fetched the webpage!
https://www.imdb.com/title/tt0050083/reviews/?ref_=tt_urv_sm
Successfully fetched the webpage!
https://www.imdb.com/title/tt0167260/reviews/?ref_=tt_urv_sm
Successfully fetched the webpage!
https://www.imdb.com/title/tt0108052/reviews/?ref_=tt_urv_sm
Successfully fetched the webpage!
https://www.imdb.com/title/tt0110912/reviews/?ref_=tt_urv_sm
Successfully fetched the webpage!
https://www.imdb.com/title/tt0120737/reviews/?ref_=tt_urv_sm
Successfully fetched the webpage!
https://www.imdb.com/title/tt0060196/reviews/?ref_=tt_urv_sm
Successfully fetched the webpage!
https://www.imdb.com/title/tt0109830/reviews/?ref_

In [83]:
reviews_df=pd.DataFrame(reviews_data1)
print(reviews_df.shape)
len(reviews_df['Movie_id'].unique())
print(reviews_df.head)

(4737, 2)
<bound method NDFrame.head of        Movie_id                                             review
0     tt0111161  It is no wonder that the film has such a high ...
1     tt0111161  I'm trying to save you money; this is the last...
2     tt0111161  This movie is not your ordinary Hollywood flic...
3     tt0111161  I have never seen such an amazing film since I...
4     tt0111161  The best movie in history and the best ending ...
...         ...                                                ...
4732  tt4430212  Movie seems foregone for all the duration of t...
4733  tt4430212  Drishyam is a Hindi remake of a Tamil monster ...
4734  tt4430212  Vijay Salgaonkar is a 4th grade dropout. He ow...
4735  tt4430212  Lazy remake. Bad re-adaptation of the original...
4736  tt4430212  If you thought Ajay Devgan and Tabu's riveting...

[4737 rows x 2 columns]>


In [ ]:
#Review scrapping code is modified by chatgpt for parllel processing to make the extraction faster.
'''
import time
import requests
from bs4 import BeautifulSoup
import concurrent.futures

reviews_data = {'Movie_id': [], 'review': []}

# Define a function to scrape reviews for a single movie
def scrape_reviews(link):
    movie_id = link.split('/')[2]
    url = "https://www.imdb.com" + link[:link.find('?')] + 'reviews/?ref_=tt_urv_sm'
    print(f"Fetching URL: {url}")
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        print(f"Successfully fetched the webpage for movie {movie_id}")
        soup = BeautifulSoup(response.content, "html.parser")
        reviews = soup.find_all('div', class_='lister-item-content')

        # Extract reviews and append to the data
        for review in reviews:
            reviews_data['Movie_id'].append(movie_id)
            x = review.find('div', class_='content').text
            reviews_data['review'].append(x[:x.find('\n', 1)].lstrip('\n'))

    else:
        print(f"Failed to retrieve webpage for movie {movie_id}. Status code: {response.status_code}")

    # Introduce a slight delay to avoid overwhelming the server
    time.sleep(2)

# Use ThreadPoolExecutor for parallel processing
def parallel_scrape(movie_links):
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        # Submit the scrape_reviews function for each movie link
        executor.map(scrape_reviews, movie_links)

# List of movie links from your movie_data DataFrame
movie_links = movie_data['Link']

# Run the scraping in parallel
parallel_scrape(movie_links)

# Check results
print(f"Scraped {len(reviews_data['review'])} reviews.")

'''


Successfully fetched the webpage for movie tt0075314
Fetching URL: https://www.imdb.com/title/tt0059578/reviews/?ref_=tt_urv_sm
Fetching URL: https://www.imdb.com/title/tt6966692/reviews/?ref_=tt_urv_sm
Successfully fetched the webpage for movie tt0993846
Fetching URL: https://www.imdb.com/title/tt0053291/reviews/?ref_=tt_urv_sm
Fetching URL: https://www.imdb.com/title/tt0120382/reviews/?ref_=tt_urv_sm
Fetching URL: https://www.imdb.com/title/tt0012349/reviews/?ref_=tt_urv_sm
Fetching URL: https://www.imdb.com/title/tt0055031/reviews/?ref_=tt_urv_sm
Successfully fetched the webpage for movie tt0059578
Successfully fetched the webpage for movie tt0372784
Fetching URL: https://www.imdb.com/title/tt10272386/reviews/?ref_=tt_urv_sm
Fetching URL: https://www.imdb.com/title/tt0042192/reviews/?ref_=tt_urv_sm
Successfully fetched the webpage for movie tt0120382
Successfully fetched the webpage for movie tt0053291
Successfully fetched the webpage for movie tt6966692
Successfully fetched the web

Fetching URL: https://www.imdb.com/title/tt0015864/reviews/?ref_=tt_urv_sm
Fetching URL: https://www.imdb.com/title/tt0198781/reviews/?ref_=tt_urv_sm
Successfully fetched the webpage for movie tt0097165
Fetching URL: https://www.imdb.com/title/tt3011894/reviews/?ref_=tt_urv_sm
Fetching URL: https://www.imdb.com/title/tt0077416/reviews/?ref_=tt_urv_sm
Fetching URL: https://www.imdb.com/title/tt0017925/reviews/?ref_=tt_urv_sm
Successfully fetched the webpage for movie tt0015864
Successfully fetched the webpage for movie tt0107207
Fetching URL: https://www.imdb.com/title/tt0015324/reviews/?ref_=tt_urv_sm
Successfully fetched the webpage for movie tt3011894
Fetching URL: https://www.imdb.com/title/tt0382932/reviews/?ref_=tt_urv_sm
Fetching URL: https://www.imdb.com/title/tt0073195/reviews/?ref_=tt_urv_sm
Successfully fetched the webpage for movie tt0198781
Successfully fetched the webpage for movie tt0017925
Successfully fetched the webpage for movie tt0077416
Successfully fetched the webp

In [51]:
reviews_df=pd.DataFrame(reviews_data)


NameError: name 'reviews_data' is not defined

In [79]:
reviews_df.to_excel('reviews.xlsx', index = False)

In [11]:
movie_metadata = pd.read_excel("/Users/hitarthparmar/Documents/Projects/IMDB_Reviews_Analysis/movies_metadata.xlsx")

In [15]:
movie_metadata.head()

,Movie_id,Title,Link,Year,Runtime,Censorship,Ratings,Rating_count,Tags,Summary,Director,Actors
0,tt0111161,The Shawshank Redemption,/title/tt0111161/?ref_=chttp_t_1,1994,142,R,9.3,3.00,"['Epic', 'Prison Drama', 'Drama', 'Back to top']",A banker convicted of uxoricide forms a friend...,"['Frank Darabont', 'Frank Darabont', 'Frank Da...","['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'..."
1,tt0068646,The Godfather,/title/tt0068646/?ref_=chttp_t_2,1972,175,R,9.2,2.10,"['Epic', 'Gangster', 'Tragedy', 'Crime', 'Dram...",The aging patriarch of an organized crime dyna...,"['Francis Ford Coppola', 'Francis Ford Coppola...","['Marlon Brando', 'Al Pacino', 'James Caan', '..."
2,tt0468569,The Dark Knight,/title/tt0468569/?ref_=chttp_t_3,2008,152,PG-13,9.0,2.90,"['Action Epic', 'Epic', 'Superhero', 'Action',...",When a menace known as the Joker wreaks havoc ...,"['Christopher Nolan', 'Christopher Nolan', 'Ch...","['Christian Bale', 'Heath Ledger', 'Aaron Eckh..."
3,tt0071562,The Godfather Part II,/title/tt0071562/?ref_=chttp_t_4,1974,202,R,9.0,1.40,"['Epic', 'Gangster', 'Tragedy', 'Crime', 'Dram...",The early life and career of Vito Corleone in ...,"['Francis Ford Coppola', 'Francis Ford Coppola...","['Al Pacino', 'Robert De Niro', 'Robert Duvall..."
4,tt0050083,12 Angry Men,/title/tt0050083/?ref_=chttp_t_5,1957,96,Approved,9.0,0.89,"['Legal Drama', 'Psychological Drama', 'Crime'...",The jury in a New York City murder trial is fr...,"['Sidney Lumet', 'Sidney Lumet', 'Sidney Lumet']","['Henry Fonda', 'Lee J. Cobb', 'Martin Balsam'..."


NameError: name 'href' is not defined